# 4. Optimizing and profiling the model

TODO: Add description

In [1]:
import timeit
import pandas as pd
import modelx as mx

In [2]:
ex1 = mx.read_model('CashValue_ME_EX1') 

In [3]:
timeit.timeit('ex1.Projection.result_pv()', globals=globals(), number=1)

1.2440227998886257

In [4]:
ex1.clear_all()

In [5]:
ex4 = mx.read_model('CashValue_ME_EX4') 

In [6]:
timeit.timeit('ex4.Projection.result_pv()', globals=globals(), number=1)

0.8405095001216978

In [7]:
ex4.clear_all()

TODO: Add description

## Compare the models with 100,000 scenarios

TODO: Add description

<div class="alert alert-warning">

**Warning:**

takes about 4GB of memory space 
    
</div>

In [8]:
ex1.Projection.scen_size = 100_000

In [9]:
timeit.timeit('ex1.Projection.result_pv()', globals=globals(), number=1)

8.051920400001109

In [10]:
ex1.clear_all()

In [11]:
ex4.Projection.scen_size = 100_000

In [12]:
timeit.timeit('ex4.Projection.result_pv()', globals=globals(), number=1)

6.547852599993348

In [13]:
ex4.clear_all()

## Profile the runs

TODO: Add description

In [14]:
def get_time_info(model):
    
    try:
        mx.start_stacktrace(maxlen=None)
        model.Projection.result_pv()
        df = pd.DataFrame.from_dict(
            mx.get_stacktrace(summarize=True), orient="index")
    finally:
        mx.stop_stacktrace()
        model.clear_all()

    # Remove model and space names from index
    prefixlen = len(model.name + '.Projection.')
    df.index = [s[prefixlen:] for s in df.index]     

    # Add duration %
    total = df['duration'].sum()
    df['dur_perc'] = df['duration'] * 100 / total
    
    return df.sort_values(['dur_perc'], ascending=False)

In [15]:
ex1.Projection.scen_size = 10000
df1 = get_time_info(ex1)
df1[['duration', 'dur_perc']].iloc[:10]

,duration,dur_perc
surr_charge_rate(t),0.624001,36.920117
inv_return_mth(t),0.269719,15.958412
premium_pp(t),0.242344,14.338736
prem_to_av_pp(t),0.056426,3.338562
pols_new_biz(t),0.053761,3.180880
"claim_pp(t, kind)",0.050353,2.979228
"av_pp_at(t, timing)",0.040608,2.402669
"pols_if_at(t, timing)",0.036391,2.153168
inv_income(t),0.031674,1.874071
pv_claims(kind),0.031282,1.850866


In [16]:
ex4.Projection.scen_size = 10000
df4 = get_time_info(ex4)
df4[['duration', 'dur_perc']].iloc[:10]

,duration,dur_perc
surr_charge_rate(t),0.545975,45.629918
inv_return_table(),0.256201,21.412039
premium_pp(t),0.107633,8.995453
pols_new_biz(t),0.027895,2.331327
inv_income_pp(t),0.027172,2.270911
surr_charge(t),0.026547,2.218706
mort_table_last_age(),0.022545,1.884170
prem_to_av_pp(t),0.020173,1.685928
"claims(t, kind)",0.019646,1.641892
"pols_if_at(t, timing)",0.019297,1.612740


In [17]:
df = pd.concat([df1[['duration']], df4[['duration']]], axis=1)
df.columns = ['df1', 'df4']
df.loc[(df['df1'] > 0.05) | (df['df4'] > 0.05)]

,df1,df4
surr_charge_rate(t),0.624001,0.545975
inv_return_mth(t),0.269719,0.000000
premium_pp(t),0.242344,0.107633
prem_to_av_pp(t),0.056426,0.020173
pols_new_biz(t),0.053761,0.027895
"claim_pp(t, kind)",0.050353,0.005729
inv_return_table(),0.015044,0.256201
